# PREPROCESAMIENTO

INDICACIÓN: El preprocesado para sacar la columna de 'Juego' lo tengo que hacer desde Google Sheets con la función de filtro y a mano, por que hay vídeos de mi canal cuyo juego no está mencionado en el título, por lo cual no puedo sacar por Jupiter Notebooks los diferentes juegos de cada una de las filas (de cada uno de los videos). A su vez, he añadido la columna SEO para poner en función de cada juego cuanto sacaría de provecho el video en funcion de la demanda y la competitividad, pero ni el primero ni este último son útiles por que no correlan.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
data = pd.read_csv('../data/raw/dataset siralatriste.csv')
data.head(1)

## SACAR LA MEDIA DE MIS VARIABLES EN FUNCIÓN DEL JUEGO

In [ ]:
#para sacar la media de compartidos de cada uno de mis juegos 
data['Med Juego Compartido'] = data.groupby('Juego')['Compartido'].transform('mean')

In [ ]:
#para sacar la media de comentarios de cada uno de mis juegos 
data['Med Juego Comentarios'] = data.groupby('Juego')['Comentarios'].transform('mean')

In [ ]:
#para sacar la media del porcentaje de likes/dislikes de cada uno de mis juegos
data['Med porcentaje likes/dislikes Juego'] = data.groupby('Juego')['Me gusta (vs. No me gusta) (%)'].transform('mean')

In [ ]:
#para sacar la media de likes de cada uno de mis juegos
data['Med Likes x juego'] = data.groupby('Juego')['Me gusta'].transform('mean')

In [ ]:
#para sacar la media de tiempo (en horas) de visualizacion de cada uno de mis juegos
data['Med Horas vistas Juego'] = data.groupby('Juego')['Tiempo de visualización (horas)'].transform('mean')

In [ ]:
#para sacar la duracion media de (en minutos) de las visualizaciones hechas en cada uno de mis juegos
data['Med Duracion de Visualizaciones Juego'] = data.groupby('Juego')['Duración media de las visualizaciones'].transform('mean')

In [ ]:
#para sacar la media de impresiones para cada uno de mis juegos
data['Med Impresiones Juego'] = data.groupby('Juego')['Impresiones'].transform('mean')

In [ ]:
#para sacar la media de porcentaje de click de impresiones para cada uno de mis juegos
data['Med porcentaje clicks Impresiones Juego'] = data.groupby('Juego')['Porcentaje de clics de las impresiones (%)'].transform('mean')

In [ ]:
#para sacar la media de visualizaciones para cada uno de mis juegos
data['Med Visualizaciones Juego'] = data.groupby('Juego')['Visualizaciones'].transform('mean')

In [ ]:
#para sacar la media de suscriptores ganados en cada uno de los juegos
data['Med Suscriptores Juego'] = data.groupby('Juego')['Suscriptores'].transform('mean')

SUSTITUIR LA VARIABLE JUEGO POR NUMEROS ENTEROS

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Juego_int'] = le.fit_transform(data['Juego'])
data.head(2)

VARIABLES NUEVAS PARA CUANDO SALGA UNA PALABRA EN LOS STRINGS DE UNA COLUMNA SEA 1 Y CUANDO NO 0

In [ ]:
data['Título_1'] = np.where(data['Título del vídeo'].str.contains("1"), 1 , 0) #PARA CUANDO UNA COLUMNA CONTIENE CIERTA PALABRA
#data[data['Título_1']==1]['Título del vídeo'].value_counts(normalize=True) #para contar dichos valores 1 o 0

In [ ]:
data['Año'] = data['Fecha'].str.split().str[-1].astype('int') #para coger solo la última palabra y hacerlo numero entero
#data.head(1)

DIVIDIR EL TIPO DE JUEGOS EN HISTORIA O NO Y PONERLE 1 ó 0

In [ ]:
data['Historia'] = np.where(data['Juego'].str.contains('Rome 2 Total War|Empire Total War|Napoleón Total War|Imperivm 3|Europa Universalis 4|Medieval Total War 2|Naval Action', regex=True), 1 , 0) #PARA CUANDO UNA COLUMNA CONTIENE CIERTA PALABRA

In [ ]:
#data[data['Historia']==1]['Juego'].value_counts(normalize=True) #para contar dichos valores 1 

DIVIDIR EL TIPO DE JUEGOS EN TOTAL WAR O NO

In [ ]:
data['TW'] = np.where(data['Juego'].str.contains('Total War'), 1 , 0)

#data[data['TW']==1]['Juego'].value_counts(normalize=True) #para contar dichos valores 1 

HABIENDO SACADO LOS AÑOS, VOY A SACAR LA MEDIA DE LAS VARIABLES QUE MÁS CORRELAN DE ESE AÑO COMO VARIABLE... SOLO SALE CON ALGO DE CORRELACIÓN LA DE VISULIZACIONES, ASÍ QUE ES LA ÚNICA QUE SE VE AQUÍ.

In [ ]:

data['Med Visualizaciones Año'] = data.groupby('Año')['Visualizaciones'].transform('mean')

[ME ESTUVO DANDO ERROR SI LO HACIA COMO LAS ANTERIORES] MEDIA DE CIERTAS VARIABLES POR VISUALIZACIÓN (AUNQUE ESTÉN CONDICIONADAS CON VISUALIZACIONES, CREO QUE SON DE LAS MEJORES VARIABLES)

In [ ]:
data['Visualizaciones necesarias para suscribirse al canal'] = data['Visualizaciones'] / data['Suscriptores']

In [ ]:
data['Visualizaciones necesarias para compartir el video'] = data['Visualizaciones'] / data['Compartido']

In [ ]:
# PARA SUSTITUIR LOS inf de las últimas dos variables por 0
data['Visualizaciones necesarias para suscribirse al canal'].replace([np.inf, -np.inf], 0, inplace=True)
data['Visualizaciones necesarias para compartir el video'].replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
data.head(40)

# REPRESENTACIÓN DE LAS CORRELACIONES

In [ ]:
plt.figure(figsize= (14,14));
sns.heatmap(data.corr().abs(), annot=True);

In [ ]:
corr = data.corr().abs() #para sacarlo con valores absolutos

In [ ]:
corr_visual = corr.loc[:,['Visualizaciones']] #para sacar las correlaciones de Visualizaciones
corr_visual.sort_values(ascending=False, by = 'Visualizaciones')

# ELIMINAR COLUMNAS INSERVIBLES Y LAS QUE NO VAYAN A PREDECIR CON 'SINCERIDAD' EL FUTURO AUNQUE TENGAN GRAN CORRELACIÓN.

ELIMINACIÓN DE VARIABLES QUE CORRELAN CON VISUALIZACIONES PERO NO SERÍAN FIELES TAN FIELES AL FUTURO

In [ ]:
data.drop(columns=['Compartido', 'Tiempo de visualización (horas)', 'Suscriptores', 'Impresiones', 'Me gusta', 'Comentarios'], inplace=True)

In [ ]:
#data.head(1)

ELIMINACIÓN DE VARIABLES QUE NO CORRELAN CON VISUALIZACIONES (incluidas las categóricas de Datos Duración media de las visualizaciones, Juego y Fecha)

In [ ]:
data.drop(columns=['SEO', 'Año', 'Juego_int', 'Título_1', 'Med Duracion de Visualizaciones Juego', 'Me gusta (vs. No me gusta) (%)', 'Med Likes x juego', 'TW', 'Duración media de las visualizaciones', 'Med porcentaje likes/dislikes Juego', 'Historia', 'Med Juego Comentarios', 'Fecha', 'Juego', 'Datos Duración media de las visualizaciones'], inplace=True)

In [ ]:
plt.figure(figsize= (12,12));
sns.heatmap(data.corr().abs(), annot=True);

In [ ]:
""" corr_visual2 = corr.loc[:,['Visualizaciones']] #para sacar las correlaciones restantes de Visualizaciones
corr_visual2.sort_values(ascending=False, by = 'Visualizaciones') """

In [ ]:
data.head(3)

# GUARDAR ARCHIVO PROCESADO.

In [ ]:
# para guardar el DataFrame como archivo CSV
data.to_csv('../data/processed.csv', index=False)